ToDo:
    - Georeferencing
    - Tests
    - Fehlermedlung der API einbauen
    - 

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import getpass
import xarray
import rasterio as rio
import os
import pandas as pd
import numpy as np
import shutil
from time import sleep
import stat
import io

directory = 'C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data'
resolution = 60    #10, 20 ,60 möglich

In [38]:
def downloadingData (aoi, collectionDate, plName, prLevel, clouds):
    '''
    Downloads the Sentinel2-Data with the given parameters
    
    Parameter: 
        aoi(str) = The type and the coordinates of the area of interest
        collectionDate = The date of the data
        plName(str) = The name of the platform
        prLevel(str) = The name of the process
        clouds = The allowed percentage of the cloudcover
    '''
    
    '''Loggin in the Scihub - API with username and password'''
    username = getpass.getpass("user: ")
    password = getpass.getpass("password: ")
    api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
    
    '''Choosing the data with bounding box (footprint), date, platformname, processinglevel and cloudcoverpercentage'''
    products = api.query(aoi, date = collectionDate, platformname = plName, processinglevel = prLevel, cloudcoverpercentage = clouds)
    
    '''Filters the products and sort the by cloudcoverpercentage'''
    products_gdf = api.to_geodataframe(products)
    products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
    
    '''Downloads the choosen files from Scihub'''
    #saveFile(products_gdf_sorted)

    products_gdf_sorted.to_csv('w')
    api.download_all(products,directory, max_attempts=10, checksum=True)

In [3]:
def listDir (path):
    '''
    Lists all files from the given directory
    
    Parameter: 
        path(str): Path to the directory
        
    Returns:
        path(str[]): An array of all filenames
    '''
    return os.listdir(path)

In [4]:
from zipfile import ZipFile
def unzip (filename):
    '''
    Unzips the file with the given filename
    
    Parameter: 
        filename(str): Name of the .zip file
    '''
    with ZipFile(os.path.join(directory, filename), 'r') as zipObj:
        zipObj.extractall(directory)

In [5]:
def delete (path):
    '''
    Deletes the file/directory with the given path
    
    Parameter: 
        path(str): Path to the file/directory
    '''
    if os.path.exists(path):
        os.remove(path)
        print("File deleted: " + path)
    else:
        print("The file does not exist")

In [6]:
def extractBands (filename,resolution):
    '''
    Extracts bandpaths from .SAFE file
    
    Parameter: 
        filename(str): Sentinel .SAFE file
        
    Returns:
        bandPaths(str[]): An array of the paths for the red and nir band
    '''
    lTwoA = listDir(os.path.join(directory, filename, "GRANULE"))
    
    if resolution == 60:
        bandName = listDir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m", str(bandName[4]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R60m", str(bandName[11]))
        bandPaths = [pathRed, pathNIR]
    
    elif resolution == 20:   
        bandName = listDir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R20m", str(bandName[9]))
        bandPaths = [pathRed, pathNIR]
    
    elif resolution == 10:
        bandName = listDir (os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m"))
        pathRed = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m", str(bandName[3]))
        pathNIR = os.path.join(directory, filename, "GRANULE", str(lTwoA[0]), "IMG_DATA", "R10m", str(bandName[4]))
        bandPaths = [pathRed, pathNIR]
    else:
        print("no such resolution")
        return -1
    
    return bandPaths

In [7]:
def loadBand (bandpath,date,tile,resolution):
    '''
    Opens and reads the red and nir band, saves them as NetCDF file
    
    Parameter: 
        bandPaths(str[]): Array with the paths to the red and nir band
    '''
    b4 = rio.open(bandpath[0])
    b8 = rio.open(bandpath[1])
    
    
    print(b8.width)
    print(b8.height)
    print(b8.bounds)
    print(b8.bounds.bottom)
    print(b8.bounds.top)
    print(b8.bounds.left)
    print(b8.bounds.right)
    print(b8.transform)
    print(b8.crs)
    print(b8.transform * (0, 0))
    print(b8.transform * (b8.width,b8.height))
    print(b8.indexes)
    print(b8.shape)
    print(b8.width)
    print(b8.width)  
    
    
       
#     with rasterio.Env(): 
    
#         profile = src.profile
    
#         profile.update(
#             dtype=rasterio.uint8,
#             count=1,
#             compress='lzw')
    
#         with rasterio.open('example.tif', 'w', **profile) as dst:
#             dst.write(array.astype(rasterio.uint8), 1)
    
    
#     new_dataset = rio.open('/tmp/new.nc',
#                                 'w',
#                                 driver='netCDF',
#                                 height=b4.shape[0],
#                                 width=b4.shape[1],
#                                 count=1,
#                                 dtype=b4.dtype,
#                                 crs='+proj=latlong',
#                                 transform=transform,)
    
#     new_dataset.write(b4, 1)
#     new_dataset.close()
    
    
    
    red = b4.read()
    nir = b8.read()
    if resolution==10:
        res=1830*3*2
    elif resolution == 20:
        res = 1830*3
    elif resolution == 60:
        res = 1830
    else:
        print("No such resolution")
        return -1
    
    j=res-1
    i=0
    lat = [0]*res
    lon = [0]*res
    while j>=0:
        lon[i]=b4.bounds.left + i*60
        lat[i]=b4.bounds.bottom + j*60
        i=i+1
        j=j-1

#     i=0
#     lon = [0]*res
#     lat = [0]*res
#     while i<res:
#         lon[i]=i
#         lat[i]=i
#         i=i+1





    time = pd.date_range(date, periods=1)
    
    dataset = xarray.Dataset(
          {
        "red": (["time","lat", "lon"], red),
        "nir": (["time","lat", "lon"], nir)
        },
         coords=dict( 
            time=time,
            lat=(["lat"], lat),
            lon=(["lon"], lon),
        ),
        
         attrs=dict(
             
             platform= plName,
             processingLevel= prLevel,
             cloudcover = clouds,
             source = "https://scihub.copernicus.eu/dhus"
         ),
    )
    
    print(dataset)



    dataset.to_netcdf(directory+"/Data"+"datacube_"+str(date)+"_"+str(tile)+"_R"+str(resolution)+".nc", 'w', format='NETCDF4')
    b4.close()
    b8.close()
    return dataset

In [8]:
def getDate(filename):
    '''
    extracts the Date of the Sentinelfilename
    Parameters:
        filename (str): name of the file
    Returns: 
        (str): Date of the File ("2020-12-31")
    '''
    return filename[11:15]+"-"+filename[15:17]+"-"+filename[17:19]

In [9]:
def getTile(filename):
    '''
    extracts the UTM-tile of the Sentinelfilename
    Parameters:
        filename (str): name of the file
    Returns: 
        (str): UTM-tile of the File ("31UMC")
    '''
    return filename[38:44]

In [10]:
def on_rm_error( func, path, exc_info):
       # path contains the path of the file that couldn't be removed
       # let's just assume that it's read-only and unlink it.
       os.chmod( path, stat.S_IWRITE )
       os.unlink( path )


Execution methods

In [41]:
'''Downloading the data'''
aoi = 'POLYGON((7.181324516246491 52.15109039913986,8.04022788336513 52.151774818649216,8.043574260120138 51.69984086555013,7.166823550308125 51.70675374223106,7.181324516246491 52.15109039913986,7.181324516246491 52.15109039913986))'
collectionDate = ('20190801', '20190815')
plName = 'Sentinel-2'
prLevel = 'Level-2A'
clouds = (0,90)

In [42]:
  downloadingData (aoi, collectionDate, plName, prLevel, clouds)

user: ········
password: ········


CancelledError: 

In [27]:
'''Unzips all data and deletes .zip file'''
for filename in os.listdir(directory):
    if filename.endswith(".zip"): 
        unzip(filename)
        delete(os.path.join(directory, filename))
        continue
    else:
        continue

File deleted: C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data\S2B_MSIL2A_20200601T103629_N0214_R008_T31UGT_20200601T135554.zip
File deleted: C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data\S2B_MSIL2A_20200601T103629_N0214_R008_T32ULC_20200601T135554.zip
File deleted: C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/Sentinel/Data\S2B_MSIL2A_20200601T103629_N0214_R008_T32UMC_20200601T135554.zip


In [28]:
'''Works with .SAFE data'''
for filename in os.listdir(directory):
    if filename.endswith(".SAFE"): 
        bandPath = extractBands(os.path.join(directory, filename),resolution)
        loadBand(bandPath,getDate(filename),getTile(filename),resolution)
        sleep(0.5)
#         shutil.rmtree(os.path.join(directory, filename) , onerror = on_rm_error ) #delete safe datei
        continue
    else:
        continue

1830
1830
BoundingBox(left=699960.0, bottom=5690220.0, right=809760.0, top=5800020.0)
5690220.0
5800020.0
699960.0
809760.0
| 60.00, 0.00, 699960.00|
| 0.00,-60.00, 5800020.00|
| 0.00, 0.00, 1.00|
EPSG:32631
(699960.0, 5800020.0)
(809760.0, 5690220.0)
(1,)
(1830, 1830)
1830
1830
<xarray.Dataset>
Dimensions:  (lat: 1830, lon: 1830, time: 1)
Coordinates:
  * time     (time) datetime64[ns] 2020-06-01
  * lat      (lat) float64 5.8e+06 5.8e+06 5.8e+06 ... 5.69e+06 5.69e+06
  * lon      (lon) float64 7e+05 7e+05 7.001e+05 ... 8.096e+05 8.097e+05
Data variables:
    red      (time, lat, lon) uint16 266 325 308 310 312 ... 454 519 622 608 424
    nir      (time, lat, lon) uint16 3443 2548 2589 2598 ... 3924 3693 3755 3835
Attributes:
    platform:         Sentinel-2
    processingLevel:  Level-2A
    cloudcover:       (0, 30)
    source:           https://scihub.copernicus.eu/dhus
1830
1830
BoundingBox(left=300000.0, bottom=5690220.0, right=409800.0, top=5800020.0)
5690220.0
5800020.0
300000.